In [1]:
!pip install -U bitsandbytes

!pip install -U transformers accelerate
#!pip install evaluate rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
# ✅ Confirm install
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

from datasets import load_from_disk
from huggingface_hub import login
#import evaluate

from google.colab import drive
import os

import pandas as pd
import re

In [4]:
root = '/content/drive'
drive.mount(root)

Mounted at /content/drive


In [5]:
os.environ['HF_TOKEN'] = '<redacted>'
login(token=os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [6]:
recipe_nlg_data_path = f'{root}/MyDrive/NLP-266/Project/RecipeNLG'

dataset = load_from_disk(f'file://{recipe_nlg_data_path}/processed_recipe_nlg_dataset')

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'ingredients', 'directions', 'source', 'NER', 'n_ingredients', 'n_steps', 'n_ner', 'domain', 'avg_step_length', 'total_step_length'],
        num_rows: 349677
    })
    validation: Dataset({
        features: ['title', 'ingredients', 'directions', 'source', 'NER', 'n_ingredients', 'n_steps', 'n_ner', 'domain', 'avg_step_length', 'total_step_length'],
        num_rows: 3670
    })
    test: Dataset({
        features: ['title', 'ingredients', 'directions', 'source', 'NER', 'n_ingredients', 'n_steps', 'n_ner', 'domain', 'avg_step_length', 'total_step_length'],
        num_rows: 3671
    })
    prompt: Dataset({
        features: ['title', 'ingredients', 'directions', 'source', 'NER', 'n_ingredients', 'n_steps', 'n_ner', 'domain', 'avg_step_length', 'total_step_length'],
        num_rows: 10000
    })
})


In [7]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)

In [8]:
model_id = "teknium/OpenHermes-2.5-Mistral-7B"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

In [10]:
def form_example(row):
    return f'''Recipe Title: {row['title']}
Ingredients: {row['ingredients']}
Your response:
'''

In [11]:
TRAIN_SIZE = 1000
VAL_SIZE = 100

train = dataset['train'].shuffle().select(range(TRAIN_SIZE)).map(lambda x: {'prompt': form_example(x)})
val = dataset['validation'].shuffle().select(range(VAL_SIZE)).map(lambda x: {'prompt': form_example(x)})

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
MAX_SEQUENCE_LENGTH = 256

def preprocess_generation_batch(batch):

    inputs_encoded = tokenizer.batch_encode_plus(
            batch["prompt"],
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

    outputs_encoded = tokenizer.batch_encode_plus(
        batch["directions"],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    return {'input_ids': inputs_encoded['input_ids'],
            'labels': outputs_encoded['input_ids']}

In [ ]:
train = train.map(preprocess_generation_batch, batched=True)
val = val.map(preprocess_generation_batch, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train.save_to_disk(f'{recipe_nlg_data_path}/train_processed_1k')
val.save_to_disk(f'{recipe_nlg_data_path}/val_processed_100')

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
train = load_from_disk(f'file://{recipe_nlg_data_path}/train_processed_1k')
val = load_from_disk(f'file://{recipe_nlg_data_path}/val_processed_100')

### Fine Tune

In [14]:
from transformers import TrainingArguments, Trainer

BATCH_SIZE = 32
NUM_EPOCHS = 5

training_args = TrainingArguments(
        "recipe_generation_model",
        eval_strategy='epoch',
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        report_to='none',
        label_names=['labels']
    )

In [15]:
!pip install -U peft

In [16]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import Trainer # Import Trainer here as well for clarity

# create LoRA configuration object
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # type of task to train on
    inference_mode=False, # set to False for training
    r=8, # dimension of the smaller matrices
    lora_alpha=32, # scaling factor
    lora_dropout=0.1  # dropout of LoRA layers
)

last_layers = list(range(28, 32))

def is_target_module(name):
    # Match q_proj or v_proj only in last 4 layers
    pattern = re.compile(r"model\.layers\.(\d+)\.self_attn\.(q_proj|v_proj)")
    match = pattern.fullmatch(name)
    if not match:
        return False
    layer_idx = int(match.group(1))
    return layer_idx in last_layers

lora_config.target_modules = [name for name, _ in model.named_modules() if is_target_module(name)]

model_peft = get_peft_model(model, lora_config)

trainable params: 425,984 || all params: 7,242,174,464 || trainable%: 0.0059


In [17]:
trainer = Trainer(
        model_peft,
        training_args,
        train_dataset=train,
        eval_dataset=val
    )

trainer.train()
model_peft.save_pretrained(f'{recipe_nlg_data_path}/qlora_1k_test_5e')

Epoch,Training Loss,Validation Loss
1,No log,6.180574
2,No log,5.616037
3,No log,5.323602
4,No log,5.126225
5,No log,5.068610


### Test Output

In [ ]:
SYSTEM_PROMPT = """
You are an expert in generating recipe instructions from the recipe title and ingredient list.
""".strip()

PROMPT_HPS = {
  'do_sample': True,
  'temperature': 0.7,
  'top_p': 1.0,
  'max_new_tokens': 256,
  'stop_sequences':None,
  'repetition_penalty': 1.1,
  'use_cache': True
}

def generate_instruction_batch(batch, model=None, tokenizer=None, cot=False, examples=None):

    prompts = [
        f"""{SYSTEM_PROMPT}
Enclose your recipe instructions in <instructions> tags and number each step.
Examples: {examples}
Title: {title}
Ingredients: {ingredients}
Your response:"""
            for title, ingredients in zip(batch['title'], batch['ingredients'])
        ]

    print(prompts[0])

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)

    hps = PROMPT_HPS.copy()

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **hps,
            pad_token_id=tokenizer.pad_token_id
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return {'generated': decoded}#[extract_text(decoded_example, "instructions") for decoded_example in decoded]}

In [ ]:
BATCH_SIZE = 2

small_test = load_from_disk(f'file://{recipe_nlg_data_path}/test_1k')
generated_dataset_oob_TEST = small_test.select([1,2]).map(
    generate_instruction_batch,
    batched=True,
    batch_size=BATCH_SIZE,
    fn_kwargs={"cot": False, "model": model_peft, "tokenizer": tokenizer}
)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


You are an expert in generating recipe instructions from the recipe title and ingredient list.
Enclose your recipe instructions in <instructions> tags and number each step.
Examples: None
Title: Tarragon Chicken Salad
Ingredients: ["1/4 cup MIRACLE WHIP FREE Dressing", "1/4 tsp. dried tarragon leaves", "3 oz. , cubed", "1 cup red or green grapes, seedless Target 1 lb For $2.99 thru 02/06", "1/2 whole wheat pita bread round", "2 SNACKWELL'S Sugar Free Shortbread Cookies", "4 fl oz (1/2 cup) orange juice"]
Your response:


In [ ]:
print(generated_dataset_oob_TEST['generated'][1])

You are an expert in generating recipe instructions from the recipe title and ingredient list.
Enclose your recipe instructions in <instructions> tags and number each step.
Examples: None
Title: Watercress Tomato Salad
Ingredients: ["8 oz. escarole, torn", "1 bunch watercress", "1 cup cherry tomato halves", "1 cup cucumbers, thinly sliced", "6 tbsp. fat-free Italian dressing", "2 tbsp. crumbled blue cheese", "1 tbsp. fat-free mayonnaise"]
Your response: <instructions>
1. Toss together the escarole, watercress, cherry tomatoes, and cucumber slices in a large bowl. <br>
2. Pour in the Italian dressing, mixing well to evenly coat the salad. <br>
3. Stir in the blue cheese and fat-free mayonnaise. <br>
4. Chill for at least 30 minutes before serving. <br>
</instructions>


In [ ]:
model_peft.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
generated_dataset_peft_TEST = small_test.select([1,2]).map(
    generate_instruction_batch,
    batched=True,
    batch_size=BATCH_SIZE,
    fn_kwargs={"cot": False, "model": model_peft, "tokenizer": tokenizer}
)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

You are an expert in generating recipe instructions from the recipe title and ingredient list.
Enclose your recipe instructions in <instructions> tags and number each step.
Examples: None
Title: Tarragon Chicken Salad
Ingredients: ["1/4 cup MIRACLE WHIP FREE Dressing", "1/4 tsp. dried tarragon leaves", "3 oz. , cubed", "1 cup red or green grapes, seedless Target 1 lb For $2.99 thru 02/06", "1/2 whole wheat pita bread round", "2 SNACKWELL'S Sugar Free Shortbread Cookies", "4 fl oz (1/2 cup) orange juice"]
Your response:


In [ ]:
print(generated_dataset_peft_TEST['generated'][0])

You are an expert in generating recipe instructions from the recipe title and ingredient list.
Enclose your recipe instructions in <instructions> tags and number each step.
Examples: None
Title: Tarragon Chicken Salad
Ingredients: ["1/4 cup MIRACLE WHIP FREE Dressing", "1/4 tsp. dried tarragon leaves", "3 oz. , cubed", "1 cup red or green grapes, seedless Target 1 lb For $2.99 thru 02/06", "1/2 whole wheat pita bread round", "2 SNACKWELL'S Sugar Free Shortbread Cookies", "4 fl oz (1/2 cup) orange juice"]
Your response: " top05 1.", lett of  using serve Ser " for SER for 6 to served with.
